In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Neural Network Beginner Template


#### some notes:
* torch.Tensor returns a float Tensor where as torch.tensor infers the type, 
* the forward function requires the data as float tensors
* when using CrossEntropyLoss the loss function requires the target values as long tensors



In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets as gen_dataset

torch.manual_seed(1)

In [ ]:
#  same as before create data set
class Dataset(BaseDataset):

    def __init__(self, num_data_points = 100, num_clusters= 1 , cluster_std = 0.1):
#         generate random clusters of data in machine learning/ AI we use X as input features or data we want to use to predict
#         y is the target values , or the values we want to predict
        self.clusters = num_clusters
        centers = [[np.random.randn(),np.random.randn()] for i in range(num_clusters) ]
        self.X,self.y = gen_dataset.make_blobs(n_samples=num_data_points,random_state=1,centers=centers,cluster_std=cluster_std)

        
    def __getitem__(self, i):
        x_data = torch.tensor(self.X[i]).float()
        y_data = torch.tensor(self.y[i])
        
        return x_data,y_data
    
    def get_all_data(self):
        return self.X,self.y
    
    def visualize(self):
        plt.figure()
        for i in range(self.clusters):
            plt.scatter(self.X[self.y==i,0],self.X[self.y==i,1] )
        plt.show()
    
    def __len__(self):
        return len(self.y)

In [ ]:
class my_model(nn.Module):
    def __init__(self, input_size = 1 , output_size = 1, num_layers = 1, width = 1):
        super().__init__()
        self.input_layer = nn.Linear(input_size,width)
        self.hidden_layer = nn.Linear(width,width)
        self.output_layer = nn.Linear(width,output_size)
        self.num_layers = num_layers
        self.output_size = output_size
        self.binary = True if self.output_size < 3 else False
        
    def forward(self,x):
        x = F.relu(self.input_layer(x))
        for i in range(self.num_layers):
            x = F.relu(self.hidden_layer(x))
            
        pred = torch.sigmoid(self.output_layer(x)) if self.binary else self.output_layer(x)
        
        return pred
    
    def predict(self,x):
        pred = self.forward(x)
        if pred.shape[1] > 2:
            return np.argmax(pred.data.numpy(),axis=1)
        else:
            return np.round(pred.detach().numpy())


In [ ]:
data = Dataset(num_data_points = 100, num_clusters= 2, cluster_std=0.7 )
data_loader = DataLoader(data ,100)
# data.visualize()


In [ ]:
%matplotlib qt


output_size = data.clusters if data.clusters > 2 else 1

model = my_model(input_size = 2 ,output_size = output_size, num_layers = 2, width = 4)

if output_size  > 2:
    criterion = nn.CrossEntropyLoss()
else:
    criterion = nn.BCELoss()
    
optimizer = torch.optim.Adam(model.parameters() , lr = 0.1)

epochs = 2000
losses = []

fig, ax = plt.subplots(2, 1)
plt.tight_layout()

for i in range(epochs):
  
    for x,y in data_loader:
       
        y_pred = model.forward(x)

        if output_size > 2:
            loss = criterion( y_pred, y )
        else:
            loss = criterion( y_pred.view(-1),y.float())
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss)

    if i%100 == 0:
        print(f'epoch:{i} loss:{loss.item()}')
#         y_pred = model.predict(x)
            
    if i%10 == 0:
                
        X = x.numpy()
        y = y.numpy()
       
        ax[0].cla()
        ax[0].set_title(f'epoch:{i}')
        h = 0.02
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

       
        Z = np.array(model.predict(torch.Tensor(np.c_[xx.ravel(), yy.ravel()])))
        Z = Z.reshape(xx.shape)

        ax[0].imshow(Z, interpolation='nearest',
        extent=(xx.min(), xx.max(), yy.min(), yy.max()),
        cmap=plt.cm.Paired,
        aspect='auto', origin='lower')

        for i in range(data.clusters):
            ax[0].scatter(X[y==i,0],X[y==i,1] )
       
        ax[1].cla()
        ax[1].set_title(f'loss:{round(loss.item(),4)}')
        ax[1].plot(np.array(losses)/len(losses) )
        plt.pause(0.1)
        
        
    